# Tratamiento de datos: transfer learning

En este notebook se detallan todos los pasos seguidos para estandarizar y analizar un conjunto de datos cualquiera, relacionado con detección de palabras de activación. Se encarga de hacer padding sobre los audios cortos (hasta que duren 1 segundo) o de seleccionar la mejor ventana de audio en audios largos, utilizando el RMS. 

Por razones de privacidad no puedo compartir el conjunto de datos recopilado para el trabajo, pero cualquier persona puede hacer su propia recopilación y subirla en su carpeta data/raw/transfer_learning, de modo que los diferentes audios estén en carpetas separadas dependiendo de la etiqueta que tengan.

Importamos los paquetes que se van a ir utilizando de forma general a lo largo del código.

In [1]:
# General libraries
import numpy as np
import pandas as pd
import random
random.seed(1997)
import os
import shutil
import json
import re

from tqdm import tqdm

# Graphics and plots
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
import plotly.express as px

# Audio specifics
import IPython
import wave
import soundfile as sf
from pydub import AudioSegment
import librosa
import librosa.display

Definimos las variables globales.

In [2]:
# Global variables - Paths
RAW_PATH = "./data/raw"
RAW_TL_PATH = os.path.join(RAW_PATH, "transfer_learning").replace("\\","/")

MODIFIED_PATH = "./data/modified"
MODIFIED_TL_PATH = os.path.join(MODIFIED_PATH, "standarised_transfer_learning").replace("\\","/")

JSON_PATH = "./data/json"

# Global variables - Others
COMMON_NFRAMES = 16000
COMMON_NFRAMES_LIBROSA = 22050

In [3]:
if not os.path.exists(RAW_TL_PATH):
    os.makedirs(RAW_TL_PATH)

if not os.path.exists(MODIFIED_TL_PATH):
    os.makedirs(MODIFIED_TL_PATH)

-----

# Datos para transfer learning

## Descripción de los datos

Una de las ventajas del transfer learning es que como el modelo base ya está entrenado, el conjunto de datos que se requiere es mucho más reducido. En nuestro caso hemos conseguido obtener audios de 102 personas diferentes. Cada persona enviaba un audio breve (de entre 1 y 3 segundos) diciendo las palabras sobre las que queremos reentrenar el modelo. En este caso hemos escogido las 18 palabras en español equivalentes al conjunto en inglés que hemos usado en el entrenamiento anterior. De este modo, el objetivo de esta parte del trabajo es investigar si el transfer learning es válido para un pequeño conjunto de comandos de activación, traduciéndolos de un lenguaje a otro.

Como hemos comentado, no es necesario un conjunto muy extenso de datos, por lo que no vamos a aplicar ninguna ténica de data augmentation o similar. Tampoco vamos a realizar un análisis exhaustivo de los datos, porque sabemos que todos los audios son válidos ya que se han ido comprobando de manera individual en el momento de la recolección. Es cierto que no son del todo homogéneos, ya que algunos presentan cierto ruido de fondo y algunas de las traducciones se convierten en palabras un poco más largas (más de 2 sílabas), lo que provoca que se tengan pronunciaciones y estiramientos de las palabras diferentes. Sin embargo, consideramos que esto otorga cierta riqueza al conjunto de datos.

Por último, comentar que los audios se han obtenido en su mayoría de personas entre los 18 y los 30 años (aunque hay un porcentaje de personas mayores de 30 años) y con una distribución más o menos del 50% entre audios obtenidos de hombres y mujeres. No se han obtenido audios ni de niños ni de personas mayores de 70 años.

*Puede ser interesante una rama del trabajo más centrada en el transfer learning para personas con peores capacidades de pronunciación.

## Estandarización de los audios brutos

El único problema con los datos brutos obtenidos es que la duración es muy irregular y la distribución de información en los audios no es homogénea. Los audios se han obtenido por WhatsApp, que no permite enviar audios de menos de 1 segundo, y todos ellos están en formato .ogg. En nuestro caso necesitamos audios exactos de 1 segundo y en formato .wav, así que es necesario estandarizar este conjunto de datos previamente, ya que si no el proceso de transfer learning puede devolver malos resultados.

El proceso que se ha seguido para la estandarización es el siguiente:

* asegurarse de la creación de las carpetas de salida
* lectura del archivo .ogg, fijando el sample rate al valor que queremos
* de manera individual se estandariza la duración de los audios:
    * en caso de que haya audios de menos de 1 segundo de duración, se ha programado una función que hace padding sobre ellos con silencio, de modo que duren finalmente 1 segundo. Esto no supone ningun problema ya que no es posible perder información importante
    * para los audios más largos de 1 segundo se ha procedido con más cuidado ya que hay riesgo de perder información. Recolectando los audios se identificaban diferentes perfiles en las grabaciones: había gente que grababa el comando y dejaba silencio al final, otros empezaban en silencio y el comando se decía justo al final, y otros decían el comando más o menos por el medio, dejando huecos de silencio al principio y al final. Esto suponía un problema ya que no se podían recortar los audios de una manera homogénea para todos los casos. Por ello, lo que se ha hecho es programar una función que se encarga de extraer el segundo con mayor información, utilizando ventanas de 1 segundo de duración y recopilando en cada caso cuál el RMS de dicha ventana. Interando sobre diferentes ventanas a lo largo de la duración del audio, nos quedamos finalmente con aquella que presenta un mayor RMS, suponiendo como hipótesis que ello indica dónde está el comando, ya que todos los audios no tenían un excesivo ruido de fondo como para influir tan significativamente en este proceso
* guardar el audio final de 1 segundo de duración en formato .wav

In [4]:
etiquetas = ["00-cero", "01-uno", "02-dos", "03-tres", "04-cuatro", "05-cinco", "06-seis", "07-siete", "08-ocho", "09-nueve", "10-aceptar", "11-rechazar", "12-arriba", "13-abajo", "14-izquierda", "15-derecha", "16-si", "17-no"]

for etiqueta in etiquetas:
    if not os.path.exists(os.path.join(MODIFIED_TL_PATH, etiqueta).replace("\\", "/")):
        os.makedirs(os.path.join(MODIFIED_TL_PATH, etiqueta).replace("\\", "/"))

In [5]:
def pad_short_audio(filename, etiqueta, number):
    pad_ms = 1000

    signal = AudioSegment.from_ogg(filename)
    assert pad_ms > len(signal), "Audio was longer that 1 second. Path: " + str(filename)
    
    silence = AudioSegment.silent(duration=pad_ms-len(signal)+1)
    padded = signal + silence  # adding silence after the signal
    out_filename = os.path.join(MODIFIED_TL_PATH, etiqueta, str(number)+".wav")
    padded.export(out_filename, format="wav")
    
    signal, sample_rate = librosa.load(out_filename, sr=COMMON_NFRAMES_LIBROSA)
    signal = signal[:COMMON_NFRAMES_LIBROSA]
    sf.write(out_filename, signal, sample_rate)

In [6]:
def extract_loudest_second(signal, sample_rate, filename, etiqueta, number):
    results = {
        "slices": [],
        "rms": []
    }

    for slice_down in range(0, len(signal)-COMMON_NFRAMES_LIBROSA, int(COMMON_NFRAMES_LIBROSA/15)): # steps of 1470 frames
        slice_up = slice_down + COMMON_NFRAMES_LIBROSA if slice_down + COMMON_NFRAMES_LIBROSA < len(signal) else len(signal) 
        results["slices"].append([slice_down, slice_up])
        results["rms"].append(np.sum(librosa.feature.rms(signal[slice_down:slice_up])))
    slice_down, slice_up = results["slices"][np.argmax(results["rms"])]

    out_filename = os.path.join(MODIFIED_TL_PATH, etiqueta, str(number)+".wav")
    signal = signal[slice_down:slice_up]
    sf.write(out_filename, signal, sample_rate)

In [7]:
for j, etiqueta in enumerate(etiquetas):
    print(etiqueta)

    for i, file in enumerate(tqdm(os.listdir(os.path.join(RAW_TL_PATH, etiqueta).replace("\\", "/")))):
        if file.endswith(".ogg"):
            filename = os.path.join(RAW_TL_PATH, etiqueta, file).replace("\\", "/")
            signal, sample_rate = librosa.load(filename, sr=COMMON_NFRAMES_LIBROSA)

            if len(signal) < COMMON_NFRAMES_LIBROSA:
                pad_short_audio(filename, etiqueta, i)
            else:
                extract_loudest_second(signal, sample_rate, filename, etiqueta, i)

00-cero


  0%|          | 0/102 [00:00<?, ?it/s]c:\Users\Carlos\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
100%|██████████| 102/102 [07:03<00:00,  4.16s/it]


01-uno


100%|██████████| 102/102 [06:53<00:00,  4.05s/it]


02-dos


...

100%|██████████| 102/102 [06:58<00:00,  4.10s/it]


16-si


100%|██████████| 102/102 [06:39<00:00,  3.91s/it]


17-no


100%|██████████| 102/102 [06:41<00:00,  3.94s/it]


Una vez terminado el proceso, se ha verificado de manera general que el procedimiento de extracción de la ventana de 1 segundo con mayor información es correcto. Lo único que se ha detectado es que hay algunos casos (muy reducidos en número) en los que el comando en sí no cabía en la ventana de un segundo y por ello ha quedado ligeramente recortado. Sin embargo esto no supone ningún problema, ya que el comando se sigue entendiendo perfectamente.

In [8]:
# To test some examples
"""
with wave.open("./data/modified/standarised_transfer_learning/10-aceptar/0.wav", mode="rb") as wav_object:
    print(wav_object.getnchannels())
    print(wav_object.getsampwidth())
    print(wav_object.getframerate())
    print(wav_object.getnframes())
"""

'\nwith wave.open("./data/modified/standarised_transfer_learning/10-aceptar/0.wav", mode="rb") as wav_object:\n    print(wav_object.getnchannels())\n    print(wav_object.getsampwidth())\n    print(wav_object.getframerate())\n    print(wav_object.getnframes())\n'

Verificamos que los nuevos audios tienen las mismas características que el conjunto de datos que hemos utilizado sobre los modelos de speech recognition, refiriéndonos al número de canales, framerate, sample width y número de frames.

In [9]:
data_TL = {
    "n_channels": [],
    "sample_width": [],
    "framerate": [],
    "n_frames": [],
    "files": [],
    "labels": []
}

for i, (dirpath, dirnames, filenames) in enumerate(os.walk(MODIFIED_TL_PATH)):

    if dirpath != MODIFIED_TL_PATH: # ensure we're at sub-folder level; if not, there are no audio files
        print(dirpath)
        for file in tqdm(filenames):
            file_full = os.path.join(dirpath, file).replace("\\","/")
            with wave.open(file_full, mode="rb") as wav_object:
                data_TL["n_channels"].append(wav_object.getnchannels())
                data_TL["sample_width"].append(wav_object.getsampwidth())
                data_TL["framerate"].append(wav_object.getframerate())
                data_TL["n_frames"].append(wav_object.getnframes())
                data_TL["files"].append(file_full)
                data_TL["labels"].append(dirpath.split("\\")[-1])
                
with open(os.path.join(JSON_PATH, "basic_wav_info_TL.json"), "w") as json_file:
    json.dump(data_TL, json_file, indent=4)

./data/modified/standarised_transfer_learning\00-cero


100%|██████████| 102/102 [00:00<00:00, 165.32it/s]


./data/modified/standarised_transfer_learning\01-uno


100%|██████████| 102/102 [00:00<00:00, 175.56it/s]


./data/modified/standarised_transfer_learning\02-dos


...

100%|██████████| 102/102 [00:00<00:00, 176.17it/s]


./data/modified/standarised_transfer_learning\16-si


100%|██████████| 102/102 [00:00<00:00, 180.53it/s]


./data/modified/standarised_transfer_learning\17-no


100%|██████████| 102/102 [00:00<00:00, 178.95it/s]


In [10]:
bad_wavs_TL = {
    "n_channels": {
        "n_channels": [],
        "files": []
    },
    "sample_width": {
        "sample_width": [],
        "files": []
    },
    "framerate": {
        "framerate": [],
        "files": []
    },
    "n_frames": {
        "n_frames": [],
        "files": []
    },
}

In [11]:
# Number of channels
EXPECTED_n_channels = 1

for i, channel in enumerate(data_TL["n_channels"]):
    if channel != EXPECTED_n_channels:
        bad_wavs_TL["n_channels"]["n_channels"].append(channel)
        bad_wavs_TL["n_channels"]["files"].append(data_TL["files"][i])

print(len(bad_wavs_TL["n_channels"]["n_channels"]))

0


In [12]:
# Sample width
EXPECTED_sample_width = 2

for i, sample_width in enumerate(data_TL["sample_width"]):
    if sample_width != EXPECTED_sample_width:
        bad_wavs_TL["sample_width"]["sample_width"].append(sample_width)
        bad_wavs_TL["sample_width"]["files"].append(data_TL["files"][i])

print(len(bad_wavs_TL["sample_width"]["sample_width"]))

0


In [13]:
# Framerate
EXPECTED_framerate = COMMON_NFRAMES_LIBROSA

for i, framerate in enumerate(data_TL["framerate"]):
    if framerate != EXPECTED_framerate:
        bad_wavs_TL["framerate"]["framerate"].append(framerate)
        bad_wavs_TL["framerate"]["files"].append(data_TL["files"][i])

print(len(bad_wavs_TL["framerate"]["framerate"]))

0


In [14]:
# Number of frames
EXPECTED_nframes = COMMON_NFRAMES_LIBROSA

for i, frames in enumerate(data_TL["n_frames"]):
    if frames != EXPECTED_nframes:
        bad_wavs_TL["n_frames"]["n_frames"].append(frames)
        bad_wavs_TL["n_frames"]["files"].append(data_TL["files"][i])

print(len(bad_wavs_TL["n_frames"]["n_frames"]))

0


Por último, una vez tenemos estandarizados todos los audios, mostramos gráficamente un resumen de los datos que disponemos para el transfer learning.

In [15]:
data_df = pd.DataFrame(columns=["num_samples", "label"])
for etiqueta in etiquetas:
    waves = [i for i in os.listdir(os.path.join(MODIFIED_TL_PATH, etiqueta)) if i.endswith(".wav")]
    data_df = data_df.append({"num_samples": len(waves), "label": etiqueta}, ignore_index = True)

# Plot
fig = px.bar(data_df, x="label", y="num_samples", labels={"label": "Etiqueta", "num_samples": "# muestras"}, title="Número de muestras por etiqueta")
fig.show()

Vemos que tenemos 102 audios de diferentes personas para cada una de las 18 etiquetas seleccionadas, de modo que tenemos un conjunto de datos perfectamente balanceado y, a priori, válido para realizar transfer learning.